# スタッキング（Stacking）
複数の異なるモデル（ベースモデル）の予測結果を、新たなモデル（メタモデル）に入力し、最終的な予測を行う方法  
異なる種類のモデルの強みを組み合わせられるため、単一モデルよりも性能が向上することがある  
複数のモデル間の相補性を活かせる

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成： y = 2*x + 1 にノイズを加える
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 2 * X + 1 + np.random.normal(0, 0.1, X.shape)

# NumPy → Tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル（1層隠れ層）
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size=16):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# シンプルな学習ループ（エポック数は100）
def train_model(model, optimizer, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(X_tensor)
        loss = criterion(preds, y_tensor)
        loss.backward()
        optimizer.step()
    return loss.item()


In [3]:
# 2つの基底モデルを学習（全データで学習）
base_model1 = SimpleMLP(hidden_size=16)
optimizer_b1 = optim.Adam(base_model1.parameters(), lr=0.01)
train_model(base_model1, optimizer_b1, epochs=100)

base_model2 = SimpleMLP(hidden_size=16)
optimizer_b2 = optim.Adam(base_model2.parameters(), lr=0.01)
train_model(base_model2, optimizer_b2, epochs=100)

# 基底モデルの予測を収集
with torch.no_grad():
    pred1 = base_model1(X_tensor)
    pred2 = base_model2(X_tensor)
# 2つの予測を横に結合：形状 (n_samples, 2)
stacked_features = torch.cat([pred1, pred2], dim=1)

# メタモデル（単純な線形回帰）
class MetaModel(nn.Module):
    def __init__(self):
        super(MetaModel, self).__init__()
        self.linear = nn.Linear(2, 1)
    def forward(self, x):
        return self.linear(x)

meta_model = MetaModel()
optimizer_meta = optim.Adam(meta_model.parameters(), lr=0.01)

# メタモデルの訓練
def train_meta(meta_model, features, target, optimizer, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        meta_model.train()
        optimizer.zero_grad()
        preds = meta_model(features)
        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
    return loss.item()

meta_loss = train_meta(meta_model, stacked_features, y_tensor, optimizer_meta, epochs=100)

# 最終予測：基底モデルの予測をメタモデルに入力
with torch.no_grad():
    meta_preds = meta_model(stacked_features)
print("Stacking ensemble final loss:", nn.MSELoss()(meta_preds, y_tensor).item())


Stacking ensemble final loss: 0.33685579895973206
